In [1]:
import json
import glob
import os
import pandas as pd

In [2]:
files = [os.path.basename(x) for x in glob.glob('./users/*.json')]
print len(files)

337442


In [3]:
def write_session_to_file(anon_id, session_count, session):
    with open('./sessions/' + anon_id + ', session_'+str(session_count)+'.json', 'w') as session_file:
            for row in session.iterrows():
                row[1].drop(['interval','timestamp_shifted','time'], inplace=True)
                if 'originalTimestamp' in row[1]: row[1].drop(['originalTimestamp'], inplace=True)
                row[1].to_json(session_file)
                session_file.write("\n")

In [4]:
for file_name in files:
    f = open('./users/%s'%file_name)
    count = 0
    context = f.readlines()
    data = []
    for i in context[:]:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]

    data = pd.DataFrame.from_records(data)
    
    data['time'] = pd.to_datetime(data['timestamp'])
    index = data.index
    data = data.sort(['time'])
    data.index = index
    data['timestamp_shifted'] = data['time'].shift()
    data['interval'] = data.apply(lambda row: (row['time'] - row['timestamp_shifted']), axis=1)
    data.set_value(0, 'interval', 0)

    beginning_of_session = 0
    session_count = 1
    anon_id = file_name.split('.')[0]
    index = 0

    for index in data.index:   
        if data.loc[index,'interval'] > pd.to_timedelta('0 days 01:00:00.00000'): 
            write_session_to_file(anon_id, session_count, data.loc[beginning_of_session:index-1])
            beginning_of_session = index
            session_count += 1

    if index >= beginning_of_session:
        write_session_to_file(anon_id, session_count, data.loc[beginning_of_session:index])
